In [54]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler,LabelEncoder,OneHotEncoder
import pickle

In [55]:
df=pd.read_csv("Churn_Modelling.csv")
df.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [56]:
df.drop(['RowNumber','CustomerId','Surname'],axis=1,inplace=True)

In [57]:
df['Geography'].unique()

array(['France', 'Spain', 'Germany'], dtype=object)

In [58]:
print(df['Geography'].head())

0    France
1     Spain
2    France
3    France
4     Spain
Name: Geography, dtype: object


In [59]:
le_gender=LabelEncoder()
df['Gender']=le_gender.fit_transform(df['Gender'])

In [60]:
ohe_geo=OneHotEncoder(sparse_output=False,drop=None)
newgeo=ohe_geo.fit_transform(df[['Geography']])

In [61]:
df1=pd.DataFrame(newgeo,columns=ohe_geo.get_feature_names_out())
df1.head()

,Geography_France,Geography_Germany,Geography_Spain
0,1.0,0.0,0.0
1,0.0,0.0,1.0
2,1.0,0.0,0.0
3,1.0,0.0,0.0
4,0.0,0.0,1.0


In [62]:
df_encoded = pd.concat([df, df1], axis=1)
df_encoded.drop(['Geography'], axis=1,inplace=True)

In [63]:
df_encoded.head()

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Geography_France,Geography_Germany,Geography_Spain
0,619,0,42,2,0.00,1,1,1,101348.88,1,1.0,0.0,0.0
1,608,0,41,1,83807.86,1,0,1,112542.58,0,0.0,0.0,1.0
2,502,0,42,8,159660.80,3,1,0,113931.57,1,1.0,0.0,0.0
3,699,0,39,1,0.00,2,0,0,93826.63,0,1.0,0.0,0.0
4,850,0,43,2,125510.82,1,1,1,79084.10,0,0.0,0.0,1.0


In [64]:
X=df_encoded.drop(['Exited'],axis=1)
y=df_encoded['Exited']
X_train,X_test,y_train,y_test=train_test_split(X,y,random_state=16,test_size=0.2)

sc=StandardScaler()
X_train=sc.fit_transform(X_train)
X_test=sc.transform(X_test)

In [65]:
pickle.dump(le_gender,open("le_gender.pkl","wb"))
pickle.dump(ohe_geo,open("ohe_geo.pkl","wb"))
pickle.dump(sc,open("sc.pkl","wb"))

In [66]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.callbacks import EarlyStopping,TensorBoard
import datetime

In [67]:
model=Sequential(
    [
        Dense(64,activation='relu',input_shape=(X_train.shape[1],)),
        Dense(32,activation='relu'),
        Dense(16,activation='relu'),
        Dense(1,activation='sigmoid')
    ]
)

model.summary()

c:\Users\rkdat\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential_3"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_13 (Dense)                │ (None, 64)             │           832 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_14 (Dense)                │ (None, 32)             │         2,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_15 (Dense)                │ (None, 16)             │           528 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_16 (Dense)                │ (None, 1)              │            17 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 3,457 (13.50 KB)

 Trainable params: 3,457 (13.50 KB)

 Non-trainable params: 0 (0.00 B)

In [68]:
optad=tf.keras.optimizers.Adam(learning_rate=0.01)
model.compile(optimizer=optad,loss="binary_crossentropy",metrics=['accuracy','precision'])

In [69]:
log_dir="logs/fit/"+ datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorflow_callback=TensorBoard(log_dir,histogram_freq=1)

In [70]:
early_stopping_callback=EarlyStopping(monitor='val_loss',patience=10,restore_best_weights=True)

In [71]:
history=model.fit(X_train,y_train,validation_data=(X_test,y_test),epochs=100,callbacks=[tensorflow_callback,early_stopping_callback])

Epoch 1/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - accuracy: 0.8059 - loss: 0.4426 - precision: 0.5641 - val_accuracy: 0.8270 - val_loss: 0.3979 - val_precision: 0.5763
Epoch 2/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.8544 - loss: 0.3616 - precision: 0.7038 - val_accuracy: 0.8545 - val_loss: 0.3526 - val_precision: 0.7163
Epoch 3/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.8559 - loss: 0.3517 - precision: 0.7598 - val_accuracy: 0.8475 - val_loss: 0.3531 - val_precision: 0.6667
Epoch 4/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.8581 - loss: 0.3429 - precision: 0.7327 - val_accuracy: 0.8565 - val_loss: 0.3477 - val_precision: 0.8058
Epoch 5/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.8595 - loss: 0.3400 - precision: 0.7647 - val_accuracy: 0.8585 - val_loss: 0.3491 - val_precision: 0.7044
Epoch 6/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.8615 - loss: 0.3430 - precision: 0.7756 - val_accuracy: 0.8575 - val

In [72]:
model.save('model.h5')

In [73]:
%load_ext tensorboard

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


In [74]:
%tensorboard --logdir logs/fit

Reusing TensorBoard on port 6006 (pid 4216), started 0:11:07 ago. (Use '!kill 4216' to kill it.)

In [32]:
import tensorflow as tf
from tensorflow.keras.models import load_model
import pickle
import pandas as pd

In [33]:
model=load_model('model.h5')
ohe_geo=pickle.load(open("ohe_geo.pkl","rb"))
le_gender=pickle.load(open("le_gender.pkl","rb"))

In [34]:
input_data={
    'CreditScore':800,
    'Geography':'Germany',
    'Gender':'Male',
    'Age':40,
    'Tenure':3,
    'Balance':75000,
    'NumOfProducts':2,
    'HasCrCard':1,
    'IsActiveMember':1,
    'EstimatedSalary':60000
}

In [35]:
input_data_df=pd.DataFrame([input_data])

In [36]:
geo_encoded=ohe_geo.transform([[input_data['Geography']]])
geo_encoded_df=pd.DataFrame(geo_encoded,columns=ohe_geo.get_feature_names_out(['Geography']))

c:\Users\rkdat\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but OneHotEncoder was fitted with feature names
  warnings.warn(


In [37]:
input_data_df=pd.concat([input_data_df.drop(['Geography'],axis=1),geo_encoded_df],axis=1)

In [40]:
input_data_df['Gender']=le_gender.transform(input_data_df['Gender'])

In [41]:
input_data_df

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Geography_France,Geography_Germany,Geography_Spain
0,800,1,40,3,75000,2,1,1,60000,0.0,1.0,0.0


In [42]:
sc=pickle.load(open('sc.pkl','rb'))

In [43]:
input_data_df=sc.transform(input_data_df)
input_data_df

array([[ 1.55507074,  0.9148601 ,  0.10982325, -0.69187418, -0.01765223,
         0.7923439 ,  0.64356595,  0.96970857, -0.70342115, -1.0148604 ,
         1.74075464, -0.56888201]])

In [47]:
pred=model.predict(input_data_df)
pred_prob=pred[0][0]
if pred_prob>0.5:
    print(f'Customer is likely to Churn with probability of {pred_prob:.3f}')
else:
    print(f'Customer is not likely to Churn with probability of {pred_prob:.3f}')

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
Customer is not likely to Churn with probability of 0.043
